# Value At Risk (VAR)

There is a trade off between risk and return 
the conclusion of MPT and CAPM

There are several measures
variance
standard deviation
total risk
beta

These measures do not consider a probability distribution

Var Is a number measure in price units
It tells you that in. large percentage of casses your portfolio is likley to not lose more than the specified amount of money

VaR measures the amount of potential loss that could happen in an investment of portfolio over a given period of time with a given degree of confidence

Easy to understand and interpret

can compare different types of assets or portfolios with VaR 

Risk Budgeting

Variance method - assumes returns are normally disributed

The desnitty of normal distribution function

https://en.wikipedia.org/wiki/Normal_distribution

Returns are normally distributed

We have to define a confidence level (95%)

z defines the number of standard deviations from the mean
z = 2.33, z = 1.64 99%, 95%



In [15]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import pandas_datareader as web
import datetime

In [16]:
def value_at_risk(pos, c, mu, sigma):
    alpha = norm.ppf(1-c)
    var = pos*(mu-sigma*alpha)
    return var

def value_at_risk_long(S,c,mu,sigma,n):
    alpha = norm.ppf(1-c)
    var = S*(mu*n-sigma*alpha,*np.sqrt(n))
    return var

In [23]:
start_date = datetime.datetime(2014,1,1)
end_date = datetime.datetime(2019,10,15)

In [24]:
citi = web.DataReader('C',data_source='yahoo', start=start_date,
                     end=end_date)

In [25]:
citi['returns'] = citi['Adj Close'].pct_change()
citi[:10]

,High,Low,Open,Close,Volume,Adj Close,returns
Date,,,,,,,
2014-01-02,52.400002,51.810001,52.029999,52.270000,16479700.0,48.062008,NaN
2014-01-03,53.470001,52.310001,52.389999,53.400002,26884900.0,49.101032,0.021618
2014-01-06,54.290001,53.430000,53.619999,53.810001,28503100.0,49.478031,0.007678
2014-01-07,54.720001,53.779999,54.590000,54.180000,28840400.0,49.818245,0.006876
2014-01-08,55.000000,54.200001,54.290001,54.810001,25989800.0,50.397526,0.011628
2014-01-09,55.279999,54.759998,55.000000,55.200001,21803700.0,50.756126,0.007115
2014-01-10,55.090000,54.299999,55.029999,54.720001,22485900.0,50.314762,-0.008696
2014-01-13,54.660000,53.439999,54.459999,53.720001,21110300.0,49.395275,-0.018275
2014-01-14,54.400002,53.630001,54.130001,53.950001,17771000.0,49.606762,0.004282


In [26]:
S = 1e6
c=0.99 # confidence is 99%
mu = np.mean(citi['returns'])  # mean
sigma = np.std(citi['returns']) # sigma is the standard deviation

In [41]:
var = value_at_risk(S,c,mu,sigma)
#Result in USD $
var

36079.532345918444

In [38]:
# lets do monte carlo simmulations
iterations = 100000
n = 1

In [39]:
class ValueAtRiskMoneteCarlo:
    def __init__(self, S, mu, sigma, c, n, iterations):
        self.S = S
        self.mu = mu
        self.sigma = sigma
        self.c = c
        self.n = n
        self.iterations = iterations
        
    def simulation(self):
        stock_data = np.zeros([self.iterations,1])
        rand = np.random.normal(0,1,[1,self.iterations])
        
        # equation for S(t) stock price
        stock_price = self.S*np.exp(self.n*(self.mu-0.5*self.sigma**2)+ self.sigma*np.sqrt(self.n)*rand)
        
        #we have to sort the stock prices to determine the percentile
        stock_price = np.sort(stock_price)
        
        #it depends on the confidence level: 95% -> 5 and 99% -> 1
        percentile = np.percentile(stock_price,(1-self.c)*100)
        
        return self.S-percentile
        

In [40]:
model = ValueAtRiskMoneteCarlo(S, mu, sigma,c,n,iterations)
r = model.simulation()
# R is the amount in USD $
r

34686.73668969667